## Training

In [ ]:
pip install torchnet

In [ ]:
import csv 
import pickle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader
from torchnet.dataset import TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/SNLP Group Project') 

In [ ]:
from models_noSP import *

## Set Device

In [ ]:
# set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available.")
else:
    device = torch.device("cpu")
    print("Using CPU.")

GPU is available.


## Load Text Features

In [ ]:
label_path = sys.path[-1]+"/Data/Labels/labels.npy"
text_path = sys.path[-1]+"/Data/Text Features/text_features.pt"
audio_path = sys.path[-1]+"/Data/Audio Features/audio_features.pt"

In [ ]:
labels = np.load(label_path)
print('Labels loaded!')

Labels loaded!


In [ ]:
text_features = torch.load(text_path, map_location=torch.device('cpu'))
print('Text features loaded!')

Text features loaded!


In [ ]:
audio_features = torch.load(audio_path, map_location=torch.device('cpu'))
print('Audio features loaded!')

Audio features loaded!


## Datasets and Loaders

In [ ]:
def train_test_split(dataset, valid_ratio, test_ratio, shuffle=True):
    test_size = int(test_ratio*len(dataset))
    valid_size = int(valid_ratio*len(dataset))
    train_size = len(dataset) - valid_size - test_size
    if shuffle:
        train_set, valid_set, test_set = data.random_split(dataset, [train_size, valid_size, test_size])
    else:
        train_idx = np.arange(train_size)
        valid_idx = np.arange(train_size, train_size+valid_size)
        test_idx = np.arange(train_size+valid_size, len(dataset))
        train_set = TensorDataset([torch.utils.data.Subset(dataset, train_idx)])
        valid_set = TensorDataset([torch.utils.data.Subset(dataset, valid_idx)])
        test_set = TensorDataset([torch.utils.data.Subset(dataset, test_idx)])
    return train_set, valid_set, test_set

In [ ]:
fold_ids = {}
for i in range(1, 6):
    fold_path = sys.path[-1]+"/Data/Folds/fold" + f"0{i}_id.txt"
    ids = []
    with open(fold_path, 'r') as f:
        for row in csv.reader(f, delimiter=' '):
            for item in row:
                if item.isdigit():
                    ids.append(int(item))
    fold_ids[f"fold0{i}"] = ids
print('Fold ids loaded!')

Fold ids loaded!


## Training

In [ ]:
def save_model(model, name):
    torch.save(model.state_dict(), name)

def load_model(model, name):
    weights = torch.load(name)
    model.load_state_dict(weights)
    return model

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

def weighted_accuracy(list_y_true, list_y_pred):
    assert (len(list_y_true) == len(list_y_pred))
    y_true = np.array(list_y_true)
    y_pred = np.array(list_y_pred)
    uwa = accuracy_score(y_true=y_true, y_pred=y_pred)
    return uwa

def unweighted_accuracy(list_y_true, list_y_pred):
    assert (len(list_y_true) == len(list_y_pred))
    y_true = np.array(list_y_true)
    y_pred = np.array(list_y_pred)
    w = np.ones(y_true.shape[0])
    for idx, i in enumerate(np.bincount(y_true)):
        w[y_true == idx] = float(1 / i)
    wa = accuracy_score(y_true=y_true, y_pred=y_pred, sample_weight=w)
    return wa

def weighted_f1(list_y_true, list_y_pred):
    assert (len(list_y_true) == len(list_y_pred))
    wf1 = f1_score(y_true=list_y_true, y_pred=list_y_pred, average='weighted')
    return wf1

def unweighted_f1(list_y_true, list_y_pred):
    assert (len(list_y_true) == len(list_y_pred))
    uwf1 = f1_score(y_true=list_y_true, y_pred=list_y_pred, average='macro')
    return uwf1

def evaluate_metrics(preds, labels):
    labels = labels.cpu().numpy().tolist()
    preds = torch.argmax(preds, dim=1).cpu().numpy().tolist()
    wa = weighted_accuracy(labels, preds)
    uwa = unweighted_accuracy(labels, preds)
    wf1 = weighted_f1(labels, preds)
    uwf1 = unweighted_f1(labels, preds)
    cm = confusion_matrix(labels, preds)
    return wa, uwa, wf1, uwf1, cm

In [ ]:
def train_model(model, params, train_loader, valid_loader, test_loader, report_loss=True):

    device = params['device']
    n_epochs = params['n_epochs']
    criterion = params['criterion']
    optimizer = params['optimizer']
    scheduler = params['scheduler']

    stop = 0
    best_valid_uwa = 0

    valid_metrics = {}
    test_metrics = {}
    train_loss, valid_loss, test_loss = [], [], []
    valid_wa, valid_uwa, valid_wf1, valid_uwf1, valid_cm = [], [], [], [], []
    test_wa, test_uwa, test_wf1, test_uwf1, test_cm = [], [], [], [], []

    for epoch in range(n_epochs):

        model.train()
        train_loss_i, batch, total = 0, 0, 0
        for data in tqdm(train_loader):
            x_text, x_audio, labels = data[0][0].to(device), data[0][1].to(device), data[0][2].to(device)
            preds, _ = model(x_text, x_audio)
            loss = criterion(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss_i += loss.item()
            batch += 1
            total += labels.shape[0]
            # if report_loss and batch%50==0:
            #     print("Batch: {}, Training Loss: {:.4f}".format(batch, train_loss_i/total))
        train_loss_i /= len(train_loader.dataset)

        valid_loss_i = 0.0
        valid_preds, valid_labels= [], []
        model.eval()
        with torch.no_grad():
            for data in tqdm(valid_loader):
                x_text, x_audio, labels = data[0][0].to(device), data[0][1].to(device), data[0][2].to(device)
                preds, _ = model(x_text, x_audio)
                loss = criterion(preds, labels)
                valid_loss_i += loss.item()
                valid_preds.append(preds)
                valid_labels.append(labels)
        valid_loss_i /= len(valid_loader.dataset)
        valid_preds = torch.cat(valid_preds)
        valid_labels = torch.cat(valid_labels)

        valid_wa_i, valid_uwa_i, valid_wf1_i, valid_uwf1_i, valid_cm_i = evaluate_metrics(valid_preds, valid_labels)

        test_loss_i = 0.0
        test_preds, test_labels= [], []
        model.eval()
        with torch.no_grad():
            for data in tqdm(test_loader):
                x_text, x_audio, labels = data[0][0].to(device), data[0][1].to(device), data[0][2].to(device)
                preds, _ = model(x_text, x_audio)
                loss = criterion(preds, labels)
                test_loss_i += loss.item()
                test_preds.append(preds)
                test_labels.append(labels)
        test_loss_i /= len(test_loader.dataset)
        test_preds = torch.cat(test_preds)
        test_labels = torch.cat(test_labels)

        test_wa_i, test_uwa_i, test_wf1_i, test_uwf1_i, test_cm_i = evaluate_metrics(test_preds, test_labels)

        scheduler.step(valid_loss_i)  # decay learning rate by validation loss

        if valid_uwa_i > best_valid_uwa:
            save_model(model, sys.path[-1]+'/Models/'+params['model_name']+'_noSP.pt')
            print("Saved model at epoch {} with validation unweighted accuracy of {:.2f}%".format(epoch+1, 100*valid_uwa_i))
            best_valid_uwa = valid_uwa_i

        if report_loss:
            print("\n")
            print("-" * 50)
            print("Epoch ", epoch+1)
            print("Training Loss: {:.4f}, Validation Loss: {:.4f}".format(train_loss_i, valid_loss_i))
            print('Validation Results: ')
            print("Weighted Accuracy: {:.4f}, Unweighted Accuracy: {:.4f}".format(valid_wa_i, valid_uwa_i))
            print("Weighted F1-Score: {:.4f}, Unweighted F1-Score: {:.4f},".format(valid_wf1_i, valid_uwf1_i))
            print('Test Results: ')
            print("Weighted Accuracy: {:.4f}, Unweighted Accuracy: {:.4f}".format(test_wa_i, test_uwa_i))
            print("Weighted F1-Score: {:.4f}, Unweighted F1-Score: {:.4f},".format(test_wf1_i, test_uwf1_i))
            print("-" * 50)
            print("\n")
        
        if len(valid_loss) > 0:  
            if valid_loss_i < valid_loss[-1]:
                stop = 0
            else:
                stop += 1
                if stop >= 10:   # stop training when no improvement observed in 10 consecutive epoches
                    print('Early stopped. No improvement in validation loss for 10 consecutive epoches.')
                    break
        
        train_loss.append(train_loss_i)
        valid_loss.append(valid_loss_i)
        test_loss.append(test_loss_i)

        valid_wa.append(valid_wa_i)
        valid_uwa.append(valid_uwa_i)
        valid_wf1.append(valid_wf1_i)
        valid_uwf1.append(valid_uwf1_i)
        valid_cm.append(valid_cm_i)

        test_wa.append(test_wa_i)
        test_uwa.append(test_uwa_i)
        test_wf1.append(test_wf1_i)
        test_uwf1.append(test_uwf1_i)
        test_cm.append(test_cm_i)

    valid_metrics['wa'] = valid_wa
    valid_metrics['uwa'] = valid_uwa
    valid_metrics['wf1'] = valid_wf1
    valid_metrics['uwf1'] = valid_uwf1
    valid_metrics['cm'] = valid_cm

    test_metrics['wa'] = test_wa
    test_metrics['uwa'] = test_uwa
    test_metrics['wf1'] = test_wf1
    test_metrics['uwf1'] = test_uwf1
    test_metrics['cm'] = test_cm

    return train_loss, valid_loss, test_loss, valid_metrics, test_metrics

## Model Parameters

In [ ]:
model_params = {'txt_embed_size': 1024,
                'embed_size': 768,
                'txt_H': 6,
                'aud_H': 6,
                'ta_H': 6,
                'at_H': 6,
                'txt_dropout': 0.1,
                'aud_dropout': 0.1,
                'ta_dropout': 0.1,
                'at_dropout': 0.1,
                'output_dim': 7}

## Loop Training

In [ ]:
for i in range(1,6):

    ids = fold_ids[f'fold0{i}']      

    dataset = TensorDataset([text_features[ids].float(), audio_features[ids].float(), torch.tensor(labels)[ids].long()])
    train_set, valid_set, test_set = train_test_split(dataset, valid_ratio=0.1, test_ratio=0.1, shuffle=False)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    valid_loader = DataLoader(valid_set, batch_size=128, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

    model_sa = SSE_Self(model_params)
    model_sa.to(device)
    criterion_sa = nn.CrossEntropyLoss()
    optimizer_sa = optim.Adam(model_sa.parameters(), lr=1e-4)
    scheduler_sa = ReduceLROnPlateau(optimizer_sa, mode='min', patience=5, factor=0.1, verbose=True)
    sa_params = {'n_epochs': 50,
                 'criterion': criterion_sa,
                 'optimizer': optimizer_sa,
                 'scheduler': scheduler_sa,
                 'model_name': f'model_sa_0{i}',
                 'device': device}
    train_loss_sa, valid_loss_sa, test_loss_sa, valid_metrics_sa, test_metrics_sa = train_model(model_sa, sa_params, train_loader, valid_loader, test_loader)
    
    with open(sys.path[-1]+f"/Training Results noSP/train_loss_sa_0{i}.pkl", 'wb') as f:
        pickle.dump(train_loss_sa, f)
    with open(sys.path[-1]+f"/Training Results noSP/valid_loss_sa_0{i}.pkl", 'wb') as f:
        pickle.dump(valid_loss_sa, f)
    with open(sys.path[-1]+f"/Training Results noSP/test_loss_sa_0{i}.pkl", 'wb') as f:
        pickle.dump(test_loss_sa, f)
    with open(sys.path[-1]+f"/Training Results noSP/valid_metrics_sa_0{i}.pkl", 'wb') as f:
        pickle.dump(valid_metrics_sa, f)
    with open(sys.path[-1]+f"/Training Results noSP/test_metrics_sa_0{i}.pkl", 'wb') as f:
        pickle.dump(test_metrics_sa, f)

    model_ca = SSE_Cross(model_params)
    model_ca.to(device)
    criterion_ca = nn.CrossEntropyLoss()
    optimizer_ca = optim.Adam(model_ca.parameters(), lr=1e-4)
    scheduler_ca = ReduceLROnPlateau(optimizer_ca, mode='min', patience=5, factor=0.1, verbose=True)
    ca_params = {'n_epochs': 50,
                'criterion': criterion_ca,
                'optimizer': optimizer_ca,
                'scheduler': scheduler_ca,
                'model_name': f'model_ca_0{i}',
                'device': device}
    train_loss_ca, valid_loss_ca, test_loss_ca, valid_metrics_ca, test_metrics_ca = train_model(model_ca, ca_params, train_loader, valid_loader, test_loader)
    
    with open(sys.path[-1]+f"/Training Results noSP/train_loss_ca_0{i}.pkl", 'wb') as f:
        pickle.dump(train_loss_ca, f)
    with open(sys.path[-1]+f"/Training Results noSP/valid_loss_ca_0{i}.pkl", 'wb') as f:
        pickle.dump(valid_loss_ca, f)
    with open(sys.path[-1]+f"/Training Results noSP/test_loss_ca_0{i}.pkl", 'wb') as f:
        pickle.dump(test_loss_ca, f)
    with open(sys.path[-1]+f"/Training Results noSP/valid_metrics_ca_0{i}.pkl", 'wb') as f:
        pickle.dump(valid_metrics_ca, f)
    with open(sys.path[-1]+f"/Training Results noSP/test_metrics_ca_0{i}.pkl", 'wb') as f:
        pickle.dump(test_metrics_ca, f)

100%|██████████| 6/6 [00:00<00:00,  7.22it/s]


Saved model at epoch 1 with validation unweighted accuracy of 35.16%


--------------------------------------------------
Epoch  1
Training Loss: 0.0542, Validation Loss: 0.0127
Validation Results: 
Weighted Accuracy: 0.2439, Unweighted Accuracy: 0.3516
Weighted F1-Score: 0.3077, Unweighted F1-Score: 0.2207,
Test Results: 
Weighted Accuracy: 0.2604, Unweighted Accuracy: 0.3583
Weighted F1-Score: 0.3060, Unweighted F1-Score: 0.2313,
--------------------------------------------------




100%|██████████| 6/6 [00:00<00:00,  7.13it/s]


Saved model at epoch 2 with validation unweighted accuracy of 42.51%


--------------------------------------------------
Epoch  2
Training Loss: 0.0469, Validation Loss: 0.0117
Validation Results: 
Weighted Accuracy: 0.3332, Unweighted Accuracy: 0.4251
Weighted F1-Score: 0.4013, Unweighted F1-Score: 0.3149,
Test Results: 
Weighted Accuracy: 0.3457, Unweighted Accuracy: 0.4439
Weighted F1-Score: 0.4165, Unweighted F1-Score: 0.3277,
--------------------------------------------------




100%|██████████| 6/6 [00:00<00:00,  6.99it/s]


...



--------------------------------------------------
Epoch  27
Training Loss: 0.0183, Validation Loss: 0.0122
Validation Results: 
Weighted Accuracy: 0.4480, Unweighted Accuracy: 0.5147
Weighted F1-Score: 0.5114, Unweighted F1-Score: 0.4542,
Test Results: 
Weighted Accuracy: 0.4477, Unweighted Accuracy: 0.5080
Weighted F1-Score: 0.5059, Unweighted F1-Score: 0.4460,
--------------------------------------------------




100%|██████████| 6/6 [00:00<00:00,  7.12it/s]




--------------------------------------------------
Epoch  28
Training Loss: 0.0182, Validation Loss: 0.0122
Validation Results: 
Weighted Accuracy: 0.4472, Unweighted Accuracy: 0.5134
Weighted F1-Score: 0.5102, Unweighted F1-Score: 0.4536,
Test Results: 
Weighted Accuracy: 0.4477, Unweighted Accuracy: 0.5080
Weighted F1-Score: 0.5059, Unweighted F1-Score: 0.4460,
--------------------------------------------------




100%|██████████| 6/6 [00:00<00:00,  7.07it/s]



--------------------------------------------------
Epoch  29
Training Loss: 0.0182, Validation Loss: 0.0122
Validation Results: 
Weighted Accuracy: 0.4472, Unweighted Accuracy: 0.5134
Weighted F1-Score: 0.5102, Unweighted F1-Score: 0.4536,
Test Results: 
Weighted Accuracy: 0.4484, Unweighted Accuracy: 0.5094
Weighted F1-Score: 0.5073, Unweighted F1-Score: 0.4469,
--------------------------------------------------


Early stopped. No improvement in validation loss for 10 consecutive epoches.
